## На этом занятии
+ Устройство Spark Dataframe API
+ Чтение данных из источника
+ Работа с данными
  - Базовый SQL
  - NA функции
  - Группировки
  - Запись данных
  - Соединения
  - Оконные функции
  - Функции pyspark.sql.functions
  - UDF функции
+ Workshop

In [1]:
# Файлы с данными
json_file = ''
output_parquet_agg = ""

## Устройство Spark Dataframe API

**Dataframe:**
+ структурированная колоночная структура данных
+ может быть создана на основе:
  - локальной коллекции
  - файла (файлов)
  - базы данных
+ в python работает значительно быстрее, чем RDD
+ под капотом использует RDD
+ позволяет выполнять произвольные SQL операции с данными
+ аналогично RDD являются ленивыми и неизменяеыми

## Из чего состоит Dataframe
+ схема [pyspsark.sql.StructType](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.StructType)
+ колонки [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column)
+ данные [pyspark.sql.Row](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Row)

In [2]:
df = spark.read.json(json_file)

In [3]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- population: long (nullable = true)



In [4]:
df.show(30, False)

+-------------------------------------+---------+-------+---------+----------+
|_corrupt_record                      |continent|country|name     |population|
+-------------------------------------+---------+-------+---------+----------+
|null                                 |Europe   |Russia |Moscow   |12380664  |
|null                                 |null     |Spain  |Madrid   |null      |
|null                                 |Europe   |France |Paris    |2196936   |
|null                                 |Europe   |Germany|Berlin   |3490105   |
|null                                 |Europe   |Spain  |Barselona|null      |
|null                                 |Africa   |Egypt  |Cairo    |11922948  |
|null                                 |Africa   |Egypt  |Cairo    |11922948  |
|{ "name":"New York, "country":"USA", |null     |null   |null     |null      |
+-------------------------------------+---------+-------+---------+----------+



In [5]:
type(df.schema)

pyspark.sql.types.StructType

In [6]:
df.columns

['_corrupt_record', 'continent', 'country', 'name', 'population']

In [7]:
from pyspark.sql.functions import col
df.select(
    col("continent"), col("country")
).show(3, False)

+---------+-------+
|continent|country|
+---------+-------+
|Europe   |Russia |
|null     |Spain  |
|Europe   |France |
+---------+-------+
only showing top 3 rows



In [8]:
df.collect()

[Row(_corrupt_record=None, continent='Europe', country='Russia', name='Moscow', population=12380664),
 Row(_corrupt_record=None, continent=None, country='Spain', name='Madrid', population=None),
 Row(_corrupt_record=None, continent='Europe', country='France', name='Paris', population=2196936),
 Row(_corrupt_record=None, continent='Europe', country='Germany', name='Berlin', population=3490105),
 Row(_corrupt_record=None, continent='Europe', country='Spain', name='Barselona', population=None),
 Row(_corrupt_record=None, continent='Africa', country='Egypt', name='Cairo', population=11922948),
 Row(_corrupt_record=None, continent='Africa', country='Egypt', name='Cairo', population=11922948),
 Row(_corrupt_record='{ "name":"New York, "country":"USA", ', continent=None, country=None, name=None, population=None)]

## Чтение данных из источника
Основной метод чтения любых источников

```df = spark.read.format(datasource_type).option(datasource_options).load(object_name)```

+ ```datasource_type``` - тип источника ("parquet", "json", "cassandra") и т. д.
+ ```datasource_options``` - опции для работы с источником (логины, пароли, адреса для подключения и т. д.)
+ ```object_name``` - имя таблицы/файла/топика/индекса

[DataframeReader](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader):
+ по умолчанию выводит схему данных
+ является трансформацией (ленивый)
+ возвращает [Dataframe](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)

### Список (неполный) поддерживаемых источников данных
+ Файлы:
  - json
  - text
  - csv
  - orc
  - parquet
+ Базы данных
  - elasticsearch
  - cassandra
  - jdbc
  - hive
+ Брокеры сообщений
  - kafka
  

**Библиотеки для работы с источниками должны быть доступны в JAVA CLASSPATH на драйвере и воркерах!**

## Работа с данными

In [9]:
df = spark.read.json(json_file)

df.show(20, False)

+-------------------------------------+---------+-------+---------+----------+
|_corrupt_record                      |continent|country|name     |population|
+-------------------------------------+---------+-------+---------+----------+
|null                                 |Europe   |Russia |Moscow   |12380664  |
|null                                 |null     |Spain  |Madrid   |null      |
|null                                 |Europe   |France |Paris    |2196936   |
|null                                 |Europe   |Germany|Berlin   |3490105   |
|null                                 |Europe   |Spain  |Barselona|null      |
|null                                 |Africa   |Egypt  |Cairo    |11922948  |
|null                                 |Africa   |Egypt  |Cairo    |11922948  |
|{ "name":"New York, "country":"USA", |null     |null   |null     |null      |
+-------------------------------------+---------+-------+---------+----------+



In [10]:
# Убираем ненужные колонки
df.drop("_corrupt_record").show(20, False)

+---------+-------+---------+----------+
|continent|country|name     |population|
+---------+-------+---------+----------+
|Europe   |Russia |Moscow   |12380664  |
|null     |Spain  |Madrid   |null      |
|Europe   |France |Paris    |2196936   |
|Europe   |Germany|Berlin   |3490105   |
|Europe   |Spain  |Barselona|null      |
|Africa   |Egypt  |Cairo    |11922948  |
|Africa   |Egypt  |Cairo    |11922948  |
|null     |null   |null     |null      |
+---------+-------+---------+----------+



In [11]:
# Удаляем дубликаты
df \
    .drop("_corrupt_record") \
    .distinct() \
    .show()

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|     null|   null|     null|      null|
|     null|  Spain|   Madrid|      null|
|   Europe|  Spain|Barselona|      null|
|   Europe| France|    Paris|   2196936|
|   Europe|Germany|   Berlin|   3490105|
|   Africa|  Egypt|    Cairo|  11922948|
|   Europe| Russia|   Moscow|  12380664|
+---------+-------+---------+----------+



In [12]:
# Удаляем пустые строки. Параметр "all" означает, что будут удалены только те строки, в которых ВСЕ элементы null
df \
    .drop("_corrupt_record") \
    .distinct() \
    .na.drop("all") \
    .show()

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|     null|  Spain|   Madrid|      null|
|   Europe|  Spain|Barselona|      null|
|   Europe| France|    Paris|   2196936|
|   Europe|Germany|   Berlin|   3490105|
|   Africa|  Egypt|    Cairo|  11922948|
|   Europe| Russia|   Moscow|  12380664|
+---------+-------+---------+----------+



In [13]:
# Заполняем пустые значения

df \
    .drop("_corrupt_record") \
    .distinct() \
    .na.drop("all") \
    .na.fill( {'continent': 'Earth', 'population': 0 } ) \
    .show()

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|    Earth|  Spain|   Madrid|         0|
|   Europe|  Spain|Barselona|         0|
|   Europe| France|    Paris|   2196936|
|   Europe|Germany|   Berlin|   3490105|
|   Africa|  Egypt|    Cairo|  11922948|
|   Europe| Russia|   Moscow|  12380664|
+---------+-------+---------+----------+



In [14]:
# Пример выборки нескольких колонок

clean_data = df \
    .drop("_corrupt_record") \
    .distinct() \
    .na.drop("all") \
    .na.fill( {'continent': 'Earth', 'population': 0 } ) \
    .select("continent", "country", "name", "population") \

clean_data.show()

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|    Earth|  Spain|   Madrid|         0|
|   Europe|  Spain|Barselona|         0|
|   Europe| France|    Paris|   2196936|
|   Europe|Germany|   Berlin|   3490105|
|   Africa|  Egypt|    Cairo|  11922948|
|   Europe| Russia|   Moscow|  12380664|
+---------+-------+---------+----------+



In [15]:
# Строим базовую группировку

clean_data.groupBy('continent').count().show(10, False)

+---------+-----+
|continent|count|
+---------+-----+
|Europe   |4    |
|Africa   |1    |
|Earth    |1    |
+---------+-----+



In [18]:
# Метод count можно спрятать внутри agg()
from pyspark.sql.functions import count

agg = clean_data.groupBy('continent').agg(count("*"))

agg.show()

+---------+--------+
|continent|count(1)|
+---------+--------+
|   Europe|       4|
|   Africa|       1|
|    Earth|       1|
+---------+--------+



In [19]:
# Чтобы колонки имели правильное имя, следует использовать метод alias()

agg = clean_data.groupBy('continent').agg(count("*").alias("count"))
agg.show()

+---------+-----+
|continent|count|
+---------+-----+
|   Europe|    4|
|   Africa|    1|
|    Earth|    1|
+---------+-----+



In [20]:
# Добавим в группировку сумму населения на каждом континенте

from pyspark.sql.functions import count, sum

agg = \
    clean_data \
    .groupBy('continent') \
    .agg(count("*").alias("city_count"), sum('population').alias("population_sum"))


agg.show()

+---------+----------+--------------+
|continent|city_count|population_sum|
+---------+----------+--------------+
|   Europe|         4|      18067705|
|   Africa|         1|      11922948|
|    Earth|         1|             0|
+---------+----------+--------------+



## Запись данных
Основной метод записи в любые системы

```df.write.format(datasource_type).options(datasource_options).mode(savemode).save(object_name)```

+ ```datasource_type``` - тип источника ("parquet", "json", "cassandra") и т. д.
+ ```datasource_options``` - опции для работы с источником (логины, пароли, адреса для подключения и т. д.)
+ ```savemode``` - режим записи данных (добавление, перезапись и т. д.)
+ ```object_name``` - имя таблицы/файла/топика/индекса

[DataFrameWriter](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter):
+ метод ```save``` является действием
+ позволяет работать с партиционированными данными (parquet, orc)
+ не всегда валидирует схему и формат данных


### Список (неполный) поддерживаемых источников данных
+ Файлы:
  - json
  - text
  - csv
  - orc
  - parquet
+ Базы данных
  - elasticsearch
  - cassandra
  - jdbc
  - hive
+ Брокеры сообщений
  - kafka
  

**Библиотеки для работы с источниками должны быть доступны в JAVA CLASSPATH на драйвере и воркерах!**



In [21]:
# Сохраним данные в parquet, предварительно отфильтровав данные

from pyspark.sql.functions import col

condition = col("continent") != "Earth"

agg \
    .filter(condition) \
    .write \
    .format("parquet") \
    .mode("overwrite") \
    .save(output_parquet_agg)

print("Ok! Data is written to {}".format(output_parquet_agg))

Ok! Data is written to hdfs:///user/atitov/agg0.parquet


In [22]:
# P.S.
# Когда мы делаем .filter в DataFrame API, мы передаем условие типа pyspark.sql.column.Column.
print(type(condition))

# когда раньше мы использовали лямбда функции в RDD, мы передавали лямбда функцию:
condition_old = lambda x: x != "Earth"
print(type(condition_old))

<class 'pyspark.sql.column.Column'>
<class 'function'>


## Соединения

Join'ы позволяют соединять два DF в один по заданным условиям.

По типу условия join'ы делятся на:
+ equ-join - соединение по равенству одного или более ключей
+ non-equ join - соединение по условию, отличному от равенства одного или более ключей

По методу соединения join'ы бывают:
![Joins](http://kirillpavlov.com/images/join-types.png)
[Источник](http://kirillpavlov.com/blog/2016/04/23/beyond-traditional-join-with-apache-spark/)

При выполнении join Spark автоматически выбирает один [из доступных алгоритмов](https://youtu.be/fp53QhSfQcI) соединения и не всегда делает это оптимально, часто применяя cross join. Поэтому, в последних версиях Spark метод ```join()``` приведет к ошибке, если под капотом он будет использовать cross join. Отключить эту проверку можно с помощью опции ```--conf spark.sql.crossJoin.enabled=true```

In [23]:
# Для демонстрации работы join используем подгтовленные данные
clean_data.printSchema()

agg = spark.read.parquet(output_parquet_agg)
agg.printSchema()

root
 |-- continent: string (nullable = false)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- population: long (nullable = false)

root
 |-- continent: string (nullable = true)
 |-- city_count: long (nullable = true)
 |-- population_sum: long (nullable = true)



In [24]:
# Самый простой join - inner join по равенству одной колонки
joined = clean_data.join(agg, 'continent', 'inner')

joined.printSchema()

joined.show()

root
 |-- continent: string (nullable = false)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- population: long (nullable = false)
 |-- city_count: long (nullable = true)
 |-- population_sum: long (nullable = true)

+---------+-------+---------+----------+----------+--------------+
|continent|country|     name|population|city_count|population_sum|
+---------+-------+---------+----------+----------+--------------+
|   Europe| Russia|   Moscow|  12380664|         4|      18067705|
|   Europe|Germany|   Berlin|   3490105|         4|      18067705|
|   Europe| France|    Paris|   2196936|         4|      18067705|
|   Europe|  Spain|Barselona|         0|         4|      18067705|
|   Africa|  Egypt|    Cairo|  11922948|         1|      11922948|
+---------+-------+---------+----------+----------+--------------+



In [25]:
# Inner join по равенству двух колонок. Поскольку двух одинаковых колонок у нас нет, мы создадим их из константы
from pyspark.sql.functions import lit

left = clean_data.withColumn("x", lit("x"))
right = agg.withColumn("x", lit("x"))

joined = left.join(right, ['continent', 'x'], 'inner')

joined.printSchema()

joined.show()

root
 |-- continent: string (nullable = false)
 |-- x: string (nullable = false)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- population: long (nullable = false)
 |-- city_count: long (nullable = true)
 |-- population_sum: long (nullable = true)

+---------+---+-------+---------+----------+----------+--------------+
|continent|  x|country|     name|population|city_count|population_sum|
+---------+---+-------+---------+----------+----------+--------------+
|   Europe|  x| Russia|   Moscow|  12380664|         4|      18067705|
|   Europe|  x|Germany|   Berlin|   3490105|         4|      18067705|
|   Europe|  x| France|    Paris|   2196936|         4|      18067705|
|   Europe|  x|  Spain|Barselona|         0|         4|      18067705|
|   Africa|  x|  Egypt|    Cairo|  11922948|         1|      11922948|
+---------+---+-------+---------+----------+----------+--------------+



In [26]:
# non-equ left join
from pyspark.sql.functions import lit

left = clean_data.withColumn("city_count_max", lit(2)).withColumnRenamed("continent", "continent_left")
right = agg.withColumnRenamed("continent", "continent_right")

join_condition = (col("continent_left") == col("continent_right")) & (col("city_count") < col("city_count_max"))

joined = left.join(right, join_condition, 'left')

joined.printSchema()

joined.show()

root
 |-- continent_left: string (nullable = false)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- population: long (nullable = false)
 |-- city_count_max: integer (nullable = false)
 |-- continent_right: string (nullable = true)
 |-- city_count: long (nullable = true)
 |-- population_sum: long (nullable = true)

+--------------+-------+---------+----------+--------------+---------------+----------+--------------+
|continent_left|country|     name|population|city_count_max|continent_right|city_count|population_sum|
+--------------+-------+---------+----------+--------------+---------------+----------+--------------+
|         Earth|  Spain|   Madrid|         0|             2|           null|      null|          null|
|        Europe|  Spain|Barselona|         0|             2|           null|      null|          null|
|        Europe| France|    Paris|   2196936|             2|           null|      null|          null|
|        Europe|Germany|   Berlin|

In [27]:
# non-equ right join
from pyspark.sql.functions import lit

left = clean_data.withColumn("city_count_max", lit(2)).withColumnRenamed("continent", "continent_left")
right = agg.withColumnRenamed("continent", "continent_right")

join_condition = (col("continent_left") == col("continent_right")) & (col("city_count") < col("city_count_max"))

joined = left.join(right, join_condition, 'right')

joined.printSchema()

joined.show()

root
 |-- continent_left: string (nullable = true)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- population: long (nullable = true)
 |-- city_count_max: integer (nullable = true)
 |-- continent_right: string (nullable = true)
 |-- city_count: long (nullable = true)
 |-- population_sum: long (nullable = true)

+--------------+-------+-----+----------+--------------+---------------+----------+--------------+
|continent_left|country| name|population|city_count_max|continent_right|city_count|population_sum|
+--------------+-------+-----+----------+--------------+---------------+----------+--------------+
|          null|   null| null|      null|          null|         Europe|         4|      18067705|
|        Africa|  Egypt|Cairo|  11922948|             2|         Africa|         1|      11922948|
+--------------+-------+-----+----------+--------------+---------------+----------+--------------+



In [28]:
# cross join
clean_data.crossJoin(agg).show(30, False)

+---------+-------+---------+----------+---------+----------+--------------+
|continent|country|name     |population|continent|city_count|population_sum|
+---------+-------+---------+----------+---------+----------+--------------+
|Earth    |Spain  |Madrid   |0         |Europe   |4         |18067705      |
|Europe   |Spain  |Barselona|0         |Europe   |4         |18067705      |
|Europe   |France |Paris    |2196936   |Europe   |4         |18067705      |
|Europe   |Germany|Berlin   |3490105   |Europe   |4         |18067705      |
|Africa   |Egypt  |Cairo    |11922948  |Europe   |4         |18067705      |
|Europe   |Russia |Moscow   |12380664  |Europe   |4         |18067705      |
|Earth    |Spain  |Madrid   |0         |Africa   |1         |11922948      |
|Europe   |Spain  |Barselona|0         |Africa   |1         |11922948      |
|Europe   |France |Paris    |2196936   |Africa   |1         |11922948      |
|Europe   |Germany|Berlin   |3490105   |Africa   |1         |11922948      |

In [29]:
# Один из самых простов вариантов ускорить работу join - сделать broadcast.
# При этом DF будет целиком склонирован на каждый воркер, что минимизирует shuffle во время выполнения join'а

from pyspark.sql.functions import broadcast

joined = clean_data.join(broadcast(agg), 'continent', 'inner')

joined.printSchema()

joined.show()

root
 |-- continent: string (nullable = false)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- population: long (nullable = false)
 |-- city_count: long (nullable = true)
 |-- population_sum: long (nullable = true)

+---------+-------+---------+----------+----------+--------------+
|continent|country|     name|population|city_count|population_sum|
+---------+-------+---------+----------+----------+--------------+
|   Europe|  Spain|Barselona|         0|         4|      18067705|
|   Europe| France|    Paris|   2196936|         4|      18067705|
|   Europe|Germany|   Berlin|   3490105|         4|      18067705|
|   Africa|  Egypt|    Cairo|  11922948|         1|      11922948|
|   Europe| Russia|   Moscow|  12380664|         4|      18067705|
+---------+-------+---------+----------+----------+--------------+



## Оконные функции

Оконные функции позволяют делать функции над "окнами" (кто бы мог подумать) данных

Окно создается из класса [pyspark.sql.Window](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Window) с указанием полей, определяющих границы окон и полей, определяющих порядок сортировки внутри окна:

```window = Window.partitionBy("a", "b").orderBy("a")```

Применяя окна, можно использовать такие полезные функции из [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions), как ```lag()``` и ```lead()```, а также эффективно работать с данными time-series, вычисляя такие параметры, как, например, среднее значение заданного поля за 3-х часовой интервал

In [30]:
# В нашем случае, используя оконные функции, мы можем построить DF из предыдущих примеров c join, 
# но без использования соединения

from pyspark.sql import Window
import pyspark.sql.functions as F

window = Window.partitionBy("continent")

agg = clean_data \
    .withColumn("city_count", F.count("*").over(window)) \
    .withColumn("population_sum", F.sum("population").over(window)) \

agg.show()

+---------+-------+---------+----------+----------+--------------+
|continent|country|     name|population|city_count|population_sum|
+---------+-------+---------+----------+----------+--------------+
|   Europe|  Spain|Barselona|         0|         4|      18067705|
|   Europe| France|    Paris|   2196936|         4|      18067705|
|   Europe|Germany|   Berlin|   3490105|         4|      18067705|
|   Europe| Russia|   Moscow|  12380664|         4|      18067705|
|   Africa|  Egypt|    Cairo|  11922948|         1|      11922948|
|    Earth|  Spain|   Madrid|         0|         1|             0|
+---------+-------+---------+----------+----------+--------------+



## Функции pyspark.sql.functions

Spark обладает достаточно большим набором встроенных функций, доступных в [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions), поэтому перед тем, как писать свою UDF, стоит обязательно поискать нужную функцию в данном пакете.

К тому же, все функции Spark принимают на вход и возвращают [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column), а это значит, что вы можете совмещать функции вместе

**Также важно помнить, что функции и колонки в Spark могут быть созданы без привязки к конкретным данным и DF**

In [31]:
from pyspark.sql.functions import *

avg_pop = \
    to_json(
        struct(
            (col("population_sum") / col("city_count")).alias("value")
        )
    ).alias("avg_pop")

agg.select(col("*"), avg_pop).show()


+---------+-------+---------+----------+----------+--------------+--------------------+
|continent|country|     name|population|city_count|population_sum|             avg_pop|
+---------+-------+---------+----------+----------+--------------+--------------------+
|   Europe|  Spain|Barselona|         0|         4|      18067705|{"value":4516926.25}|
|   Europe| France|    Paris|   2196936|         4|      18067705|{"value":4516926.25}|
|   Europe|Germany|   Berlin|   3490105|         4|      18067705|{"value":4516926.25}|
|   Europe| Russia|   Moscow|  12380664|         4|      18067705|{"value":4516926.25}|
|   Africa|  Egypt|    Cairo|  11922948|         1|      11922948|{"value":1.192294...|
|    Earth|  Spain|   Madrid|         0|         1|             0|       {"value":0.0}|
+---------+-------+---------+----------+----------+--------------+--------------------+



In [32]:
# Большим преимуществом Spark по сравнению с большинством SQL ориентированных БД является наличие
# встроенных функций работы со списками, словарями и структурами данных

from pyspark.sql.functions import *

all_in_one = agg.select(struct(*agg.columns).alias("allinone"))

all_in_one.printSchema()
all_in_one.show(20, False)

root
 |-- allinone: struct (nullable = false)
 |    |-- continent: string (nullable = false)
 |    |-- country: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- population: long (nullable = false)
 |    |-- city_count: long (nullable = false)
 |    |-- population_sum: long (nullable = true)

+-----------------------------------------------+
|allinone                                       |
+-----------------------------------------------+
|[Europe, Spain, Barselona, 0, 4, 18067705]     |
|[Europe, France, Paris, 2196936, 4, 18067705]  |
|[Europe, Germany, Berlin, 3490105, 4, 18067705]|
|[Europe, Russia, Moscow, 12380664, 4, 18067705]|
|[Africa, Egypt, Cairo, 11922948, 1, 11922948]  |
|[Earth, Spain, Madrid, 0, 1, 0]                |
+-----------------------------------------------+



In [33]:
# Например, можно создавать массивы и искать в них данные

from pyspark.sql.functions import *

arrays = \
    spark.range(0,1) \
    .withColumn("a", array(lit(1), lit(2), lit(3))) \
    .select(array_contains(col("a"), 1).alias("c"))


arrays.show(1, False)

+----+
|c   |
+----+
|true|
+----+



Также, в разделе [SQL, Built-in Functions](https://spark.apache.org/docs/latest/api/sql/index.html) присутствует еще более широкий список функций, доступных в Spark. Некоторые из них отсутствуют в [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions)! 

Эти функции нельзя использовать как обычные методы над [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column), однако вы можете использовать метод ```expr()``` для этого.

In [34]:
# В данном примере мы используем Java функцию с помощью функции java_method
# Запомните этот пример и используйте всегда, когда вам не хватает какой-либо функции в pyspark, 
# доступной в Java, ведь, используя такой подход, вы не снижаете производительность вашей программы за счет
# передачи данных между Python и JVM приложением Spark, и при этом вам не нужно уметь писать код на Java/Scala :)

from pyspark.sql.functions import *

spark.range(0,1).withColumn("a", expr("java_method('java.util.UUID', 'randomUUID')")).show(1, False)

+---+------------------------------------+
|id |a                                   |
+---+------------------------------------+
|0  |7afcc18f-61c4-416a-9cfe-cb9a174d0c87|
+---+------------------------------------+



## UDF функции

В том случае, когда вам не хватает функционала, описанного выше, и требуется создать специфичную функцию, вы можете использовать UDF. 

в pyspark UDF бывают трех видов:
+ native pyspark udf
+ pandas udf
+ java udf

### native pyspark udf
+ Низкая скорость из-за накладных расходов, связанных с передачей данных между Python и JVM
+ По своей сути мало чем отличаются от RDD в pyspark
+ Использование данных функций следует избегать всегда, когда это возможно

In [35]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf, lit, expr
slen = udf(lambda s: len(s), IntegerType())

_ = spark.udf.register("slen", slen)

udf_data = \
    spark.range(0,1) \
    .select(lit("aaa").alias("a")) \
    .select(expr("slen(a)"))

udf_data.show()

+-------+
|slen(a)|
+-------+
|      3|
+-------+



### pandas udf
+ Средняя скорость работы из-за частично решенной, но все еще присутсвующей проблемы с сериализацией
+ Называется pandas, т.к. на вход функции подаются не скалярные данные, как в обычной функции, а ```pandas.Series``` вектора
+ Для использования данных функций необходимо включить опцию ```--conf spark.sql.execution.arrow.enabled=true```, т.к. это позволяет существенно сократить время, требуемое для копирования массивов данных в RAM между разными структурами
+ Если вам необходимо писать UDF на python, то, на текущий момент, pandas_udf - это единственный способ, как это сделать
+ Слабая поддержка вложенных структур и массивов

In [36]:
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import IntegerType

def multiply_func(a, b):
    return a + b

multiply = pandas_udf(multiply_func, returnType=IntegerType())

spark.range(0,10).select(multiply(col("id"), col("id"))).show()

+---------------------+
|multiply_func(id, id)|
+---------------------+
|                    0|
|                    2|
|                    4|
|                    6|
|                    8|
|                   10|
|                   12|
|                   14|
|                   16|
|                   18|
+---------------------+



In [37]:
from pyspark.sql.functions import lit, expr
from pyspark.sql.types import BooleanType

spark.udf.registerJavaFunction("starts_with_e", "local.spark.udf.TestUDF", BooleanType())

spark.range(0,1).withColumn("id", lit("Egypt")).select(expr("starts_with_e(id)")).show()

+---------------------+
|UDF:starts_with_e(id)|
+---------------------+
|                 true|
+---------------------+

